# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Копин Борис Александрович

Группа: 

In [1]:
import numpy as np
import time
import pandas as pd
from sklearn import metrics
from sklearn import neighbors
from sklearn.metrics import f1_score

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
features = X_train.columns

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния (учитите, что он должен возвращать вероятность — отношение объектов первого класса среди соседей к числу соседей). Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba).

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

In [7]:
from collections import Counter


class KNeighborsClassifier(object):
    
    def __init__(self, dist_func='overlap', feature_weights=None, **kwargs):
        self.dist_func = dist_func
        
    def fit(self, X, y=None, feature_weights=None):
        """
        Parameters
        ----------
        X : array, shape (n_objects, n_features)
            List of n_features-dimensional data points.  Each row
            corresponds to a single data point.
        """
        self._fit_X = X
        self._fit_y = y
        self.classes = set(y)
        self.feature_weights = feature_weights or np.ones(X.shape[1])
        self._sorted_neighbors = None
    
    
    def fit_with_pred(self, X):
        print "Find coincidence features"
        coincidence_matrix = X[:,np.newaxis]==self._fit_X[np.newaxis,:]
        
        print "Compute dist(self._fit_X, X)"
        dist_func = _dist_funcs[self.dist_func]
        dist_matrix = dist_func(self, X, coincidence_matrix)
        
        print "Get class indices"
        nearest_X_fit_indices = np.argsort(dist_matrix, axis=1)
        
        print "Sort and save neighbors"
        self._sorted_neighbors_idxs = self._fit_y[nearest_X_fit_indices]
        
    
    def predict(self, k):
        k_nearest_neighbors = self._sorted_neighbors_idxs[:, :k]
        
        score = np.array([np.sum(k_nearest_neighbors == c, axis=1) for c in self.classes]).T
        X_y = np.argmax(score, axis=1)
        
        return X_y
    
    def _load_sorted_neighbors_idxs(self, sorted_neighbors_idxs):
        self._sorted_neighbors_idxs = sorted_neighbors_idxs


def _overlap_dist(KNN, X, coincidence_matrix):
    print "_overlap_dist"
    weighted_coincidence_matrix = coincidence_matrix * KNN.feature_weights
    
    print "Sum Feature distance"
    dist_matrix = np.sum(weighted_coincidence_matrix, axis=2)
    
    return dist_matrix


def _get_accumulated_p_2(f, l):
    print "_get_accumulated_p_2"
    p   = [Counter({v: f_v / float(l) for v, f_v in c.items()}) for c in f]
    p_2 = [Counter({v: f_v*(f_v - 1)/float(l*(l-1)) for v, f_v in c.items()}) for c in f]
    
    A_dtype=[('x', int), ('p', float), ('p2', float)]
    A_order = 'p'
    A = []
    for i, c in enumerate(f):
        u = np.array([(k, p[i].get(k), p_2[i].get(k)) for k in c.keys()], dtype=A_dtype)
        u.sort(order=A_order)
        A.append(np.array(u.tolist()[::-1]))

    # cumulative sum over p_2
    B = [np.column_stack( [ f_A , np.cumsum(f_A, axis=0)[:,2] ] ) for f_A in A] 
    
    # extract p_2 only
    P_2 = [Counter({int(x):f_B[i, 3] for i, x in enumerate(f_B[:,0])}) for f_B in B]
    
    return P_2

def _smooth_overlap_dist(KNN, X, coincidence_matrix):
    print "_smooth_overlap_dist"
    f = [Counter(c) for c in KNN._fit_X.T]
    
    accum_p_2 = _get_accumulated_p_2(f, KNN._fit_X.shape[0])
    
    # Extend each X 
    print "Extend each X "
    X_accum_p_2 = [[accum_p_2[i].get(feature) for i, feature in enumerate(x)] for x in KNN._fit_X]
    X_accum_p_2 = np.array(X_accum_p_2)
    
    print "Get P_2"
    idx_coincidence_features = np.nonzero(coincidence_matrix)
    X_coincidence_idx = (idx_coincidence_features[1], idx_coincidence_features[2])
    
    print "Feature distance"
    dist_matrix = np.ones((X.shape[0], KNN._fit_X.shape[0], KNN._fit_X.shape[1]))
    dist_matrix[idx_coincidence_features] = X_accum_p_2[X_coincidence_idx]
    
    print "Sum Feature distance"
    dist_matrix_sum = np.sum(dist_matrix, axis=2)
        
    return dist_matrix_sum

def _map_to_log_f(x):
    print "_map_to_log_f"
    f_x = [Counter(c) for c in x.T]
    log_f_x = [Counter({v: np.log(c) for v, c in f.items()}) for f in f_x]
    x_log_f = [[log_f_x[f_i].get(f) for f_i, f in enumerate(obj)] for obj in x]
    return np.array(x_log_f)
    
def _type3_dist(KNN, X, coincidence_matrix):
    print "_type3_dist"
    fit_X_log_f = _map_to_log_f(KNN._fit_X)
    X_log_f = _map_to_log_f(X)
    
    print "np.dot(X_log_f, fit_X_log_f.T)"
    a_b_f = np.dot(X_log_f, fit_X_log_f.T)

    x = KNN._fit_X
    y = X
    A = coincidence_matrix

    print "Step 1"
    A_x = A.reshape(y.shape[0], x.size)*fit_X_log_f.reshape(1, x.size)
    A_x = A_x.reshape(y.shape[0], x.shape[0], x.shape[1])

    print "Step 2"
    A_x_T = A_x.swapaxes(0, 1)
    A_x_T_slab = A_x_T.reshape(len(x), len(y)*x.shape[1])

    print "Step 3"
    A_x_y_T_slab = A_x_T_slab * X_log_f.reshape(1, y.size)

    print "Step 4"
    A_x_y_T = A_x_y_T_slab.reshape(len(x), len(y), x.shape[1])
    A_x_y = A_x_y_T.swapaxes(1, 0)

    print "Step 5"
    A_x_y_sum = np.sum(A_x_y, axis=2)

    print "Step 6"
    dist_matrix = a_b_f - A_x_y_sum
    
    return dist_matrix
    
    

_dist_funcs = {
    'overlap': _overlap_dist,
    'smooth_overlap': _smooth_overlap_dist,
    'type3': _type3_dist,
}

## Overlap dist

In [14]:
knn_overlap = KNeighborsClassifier(dist_func="overlap")
knn_overlap.fit(X_train.as_matrix(), y_train.as_matrix())

In [15]:
%%timeit -n1 -r1
knn_overlap.fit_with_pred(X_test.as_matrix())

Find coincidence features
Compute dist(self._fit_X, X)
_overlap_dist
Sum Feature distance
Get class indices
Sort and save neighbors
1 loops, best of 1: 1min 24s per loop


In [16]:
y_pred = knn_overlap.predict(k=10)
f1_score(y_test, y_pred)

0.96836140406107352

In [ ]:
# Save overlap_neighbors_idxs
np.savez("overlap_neighbors_idxs.dat",
         overlap_neighbors_idxs=knn_overlap._sorted_neighbors_idxs)

## Smooth overlap

In [11]:
knn_smooth_overlap = KNeighborsClassifier(dist_func="smooth_overlap")
knn_smooth_overlap.fit(X_train.as_matrix(), y_train.as_matrix())

In [12]:
%%timeit -n1 -r1
knn_smooth_overlap.fit_with_pred(X_test.as_matrix())

Find coincidence features
Compute dist(self._fit_X, X)
_smooth_overlap_dist
_get_accumulated_p_2
Extend each X 
Get P_2
Feature distance
Sum Feature distance
Get class indices
Sort and save neighbors
1 loops, best of 1: 2min 56s per loop


In [13]:
y_pred = knn_smooth_overlap.predict(k=10)
f1_score(y_test, y_pred)

0.9711824755423224

In [ ]:
# Save overlap_neighbors_idxs
np.savez("smooth_overlap_neighbors_idxs.dat",
         smooth_overlap_neighbors_idxs=knn_smooth_overlap._sorted_neighbors_idxs)

## Type3

In [17]:
knn_type3 = KNeighborsClassifier(dist_func="type3")
knn_type3.fit(X_train.as_matrix(), y_train.as_matrix())

In [18]:
%%timeit -n1 -r1
knn_type3.fit_with_pred(X_test.as_matrix())

Find coincidence features
Compute dist(self._fit_X, X)
_type3_dist
_map_to_log_f
_map_to_log_f
np.dot(X_log_f, fit_X_log_f.T)
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Get class indices
Sort and save neighbors
1 loops, best of 1: 3min 51s per loop


In [19]:
y_pred = knn_type3.predict(k=10)
f1_score(y_test, y_pred)

0.97050225883603514

In [14]:
# Save overlap_neighbors_idxs
np.savez("type3_neighbors_idxs.dat",
         type3_neighbors_idxs=knn_type3._sorted_neighbors_idxs)

### Restore overlap_neighbors_idxs

In [ ]:
overlap_neighbors_idxs = np.load('overlap_neighbors_idxs.dat.npz')['overlap_neighbors_idxs']

In [ ]:
knn = KNeighborsClassifier(dist_func="overlap")
knn.fit(X_train.as_matrix(), y_train.as_matrix())
knn._load_sorted_neighbors_idxs(overlap_neighbors_idxs)

In [ ]:
y_pred = knn.predict(k=10)
f1_score(y_test, y_pred)

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?